In [20]:
import pandas as pd
import numpy as np
data = {
    "Aspecto": ["Sol", "Sol", "Nuvens", "Chuva", "Chuva", "Chuva", "Nuvens", "Sol", "Sol", "Chuva", "Sol", "Nuvens", "Nuvens", "Chuva"],
    "Temp": ["Quente", "Quente", "Quente", "Ameno", "Fresco", "Fresco", "Fresco", "Ameno", "Fresco", "Ameno", "Ameno", "Quente", "Quente", "Ameno"],
    "Humidade": ["Elevada", "Elevada", "Elevada", "Elevada", "Normal", "Normal", "Normal", "Normal", "Normal", "Elevada", "Normal", "Elevada", "Normal", "Elevada"],
    "Vento": ["Fraco", "Forte", "Fraco", "Fraco", "Fraco", "Forte", "Fraco", "Forte", "Fraco", "Fraco", "Forte", "Fraco", "Forte", "Forte"],
    "Tenis": ["Não", "Não", "Sim", "Sim", "Sim", "Não", "Sim", "Não", "Sim", "Sim", "Sim", "Sim", "Sim", "Não"]
}
df = pd.DataFrame(data)
X = df.drop(columns=['Tenis'])  
y = df['Tenis']
display(df)

,Aspecto,Temp,Humidade,Vento,Tenis
0,Sol,Quente,Elevada,Fraco,Não
1,Sol,Quente,Elevada,Forte,Não
2,Nuvens,Quente,Elevada,Fraco,Sim
3,Chuva,Ameno,Elevada,Fraco,Sim
4,Chuva,Fresco,Normal,Fraco,Sim
5,Chuva,Fresco,Normal,Forte,Não
6,Nuvens,Fresco,Normal,Fraco,Sim
7,Sol,Ameno,Normal,Forte,Não
8,Sol,Fresco,Normal,Fraco,Sim
9,Chuva,Ameno,Elevada,Fraco,Sim


In [21]:
def entropia(col):
    counts = np.unique(col, return_counts=True)
    N = float(col.shape[0])
    ent = 0.0
    for ix in counts[1]:
        p = ix / N
        ent += -1.0 * p * np.log2(p)
    return ent

In [22]:

def information_gain(df, attr, target):
    total = entropia(df[target])
    valores = np.unique(df[attr])
    acc = 0
    for v in valores:
        subset = df[df[attr] == v][target]
        x = len(subset) / len(df)
        acc += x * entropia(subset)
    return total - acc

In [25]:
class DecisionTreeCategorical:
    def __init__(self, depth=0, max_depth=3):
        self.children = {}
        self.attr = None
        self.max_depth = max_depth
        self.depth = depth
        self.target = None

    def train(self, df, features, target):
        # Se todos são iguais, ou acabou feature, vira folha
        if len(np.unique(df[target])) == 1 or len(features) == 0 or self.depth >= self.max_depth:
            self.target = df[target].mode()[0]
            return

        # Escolhe melhor atributo
        gains = [information_gain(df, attr, target) for attr in features]
        best_attr = features[np.argmax(gains)]
        self.attr = best_attr
        self.children = {}

        for v in np.unique(df[best_attr]):
            subset = df[df[best_attr] == v]
            if subset.empty:
                self.children[v] = None
            else:
                child = DecisionTreeCategorical(depth=self.depth+1, max_depth=self.max_depth) #Aqui definimos Child como objeto
                child.train(subset, [f for f in features if f != best_attr], target)
                self.children[v] = child #Para o valor v do atributo, o filho correspondente é o objeto child.

    def predict(self, row):
        if self.attr is None or self.children == {}:
            return self.target
        val = row[self.attr]
        if val in self.children and self.children[val] is not None:
            return self.children[val].predict(row)
        else:
            return self.target  # fallback para modo do nó

In [26]:
for i in range(len(df)):
    test_row = df.iloc[i]
    print(f"Exemplo {i+1}: Previsto={tree.predict(test_row)}, Real={test_row[target]}")

Exemplo 1: Previsto=Não, Real=Não
Exemplo 2: Previsto=Não, Real=Não
Exemplo 3: Previsto=Sim, Real=Sim
Exemplo 4: Previsto=Sim, Real=Sim
Exemplo 5: Previsto=Sim, Real=Sim
Exemplo 6: Previsto=Não, Real=Não
Exemplo 7: Previsto=Sim, Real=Sim
Exemplo 8: Previsto=Não, Real=Não
Exemplo 9: Previsto=Sim, Real=Sim
Exemplo 10: Previsto=Sim, Real=Sim
Exemplo 11: Previsto=Não, Real=Sim
Exemplo 12: Previsto=Sim, Real=Sim
Exemplo 13: Previsto=Sim, Real=Sim
Exemplo 14: Previsto=Não, Real=Não
